
# P1 FEM for 1D Poisson (`-u'' = f` on (0,1) with Dirichlet BCs)

This notebook reproduces the classic MATLAB demo using Python/NumPy/SciPy/Matplotlib.

We solve
\[-u''(x)=f(x)\ \text{on } (0,1), \qquad u(0)=u(1)=0,\]
with \( f(x)=1 \), whose exact solution is \( u(x)=\tfrac12(x-x^2) \).

What you'll see:
1. The exact solution.
2. A few P1-FEM overlays on progressively refined uniform meshes (hidden after a short pause, like the MATLAB animation).
3. A convergence study: \(\|u-u_h\|_{L^2(0,1)}\) vs. mesh size \(h\) on a log–log plot.


In [ ]:

# Imports and plotting setup
# Notes:
# - Requires SciPy for sparse linear algebra.
# - Matplotlib is used for plots (no styles/colors forced).
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

# (Optional) nicer printing; not strictly necessary
np.set_printoptions(suppress=True)

# Make plots appear inline in Jupyter
%matplotlib inline


In [ ]:

# Problem data and helpers

def f(x):
    """Forcing term f(x) = 1."""
    return np.ones_like(x)

def u_exact(x):
    """Exact solution: u(x) = 0.5 * (x - x**2)."""
    return 0.5*(x - x**2)

# 5-point Gauss–Legendre quadrature nodes/weights on [-1,1]
g_xi = np.array([
    0.0,
    -0.5384693101056831,  0.5384693101056831,
    -0.9061798459386640,  0.9061798459386640
])
g_w  = np.array([
    0.5688888888888889,
    0.4786286704993665,  0.4786286704993665,
    0.2369268850561891,  0.2369268850561891
])


In [ ]:

# Plot the exact solution
t = np.linspace(0.0, 1.0, 10001)
plt.figure()
plt.plot(t, u_exact(t), linewidth=2, label="exact")
plt.xlabel(r"$x$")
plt.ylabel(r"$u(x)$")
plt.title("Exact solution")
plt.axis([0, 1, 0, 1.2*np.max(u_exact(t))])
plt.legend()
plt.show()


In [ ]:

# P1 FEM on uniform meshes, with optional overlays for a few refinements
num_levels = 12            # number of uniform refinement levels
show_overlays = True       # set False to skip overlay animation
overlay_levels = 4         # show overlays for the first few levels

err = np.zeros(num_levels) # L2 error for each level
hvals = np.zeros(num_levels)

# Prepare a figure for overlays (exact + a few FE lines)
plt.figure()
plt.plot(t, u_exact(t), linewidth=2, label="exact")
plt.xlabel(r"$x$")
plt.ylabel(r"$u(x)$")
plt.title("Exact solution and P1-FEM overlays")
plt.axis([0, 1, 0, 1.2*np.max(u_exact(t))])

for k in range(1, num_levels+1):
    # --- Grid and mesh size ---
    N = 1 + 2**k                # number of grid points
    h = 1.0/(N-1)               # uniform mesh size
    hvals[k-1] = h
    x = np.linspace(0.0, 1.0, N)  # node coordinates

    # --- Assemble stiffness matrix for interior nodes (Dirichlet BCs) ---
    Ndof = N - 2                 # number of interior unknowns
    if Ndof > 0:
        main_diag = (2.0/h) * np.ones(Ndof)
        off_diag  = (-1.0/h) * np.ones(Ndof-1) if Ndof > 1 else np.array([])
        K = diags([off_diag, main_diag, off_diag], offsets=[-1, 0, 1], format="csc")

        # --- Right-hand side using nodal interpolant of f (matches MATLAB script) ---
        rhs = h * f(x[1:-1])

        # --- Solve K * uh_int = rhs ---
        uh_int = spsolve(K, rhs)

        # --- Full solution including boundary zeros ---
        uh = np.zeros(N)
        uh[1:-1] = uh_int
    else:
        # For degenerate case (shouldn't happen here)
        uh = np.zeros(N)

    # --- Optional overlays for the first few refinement levels ---
    if show_overlays and k <= overlay_levels:
        line, = plt.plot(x, uh, linewidth=1, label=f"FE k={k}")
        plt.legend()
        plt.pause(0.7)   # brief pause to visualize
        # hide the line to emulate MATLAB demo behavior
        line.set_visible(False)
        plt.pause(0.1)

    # --- Compute L2 error via 5-pt Gauss quadrature per element ---
    e2 = 0.0
    for j in range(N-1):
        xL, xR = x[j], x[j+1]
        hj = xR - xL
        slope = (uh[j+1] - uh[j]) / hj
        # Map gauss points from [-1,1] to [xL,xR]: x = xm + xr * xi
        xm = 0.5*(xR + xL)
        xr = 0.5*hj
        xq = xm + xr * g_xi
        uhq = uh[j] + slope * (xq - xL)
        rq = u_exact(xq) - uhq
        e2 += np.sum(g_w * (rq**2)) * xr
    err[k-1] = np.sqrt(e2)

plt.show()

# Show a small convergence table (h, error, and estimated rate between levels)
rates = np.full(num_levels, np.nan)
for i in range(1, num_levels):
    rates[i] = np.log(err[i-1]/err[i]) / np.log(hvals[i-1]/hvals[i])

import pandas as pd
from caas_jupyter_tools import display_dataframe_to_user

df = pd.DataFrame({
    "level k": np.arange(1, num_levels+1),
    "N points": 1 + 2**np.arange(1, num_levels+1),
    "h": hvals,
    "L2 error": err,
    "rate (≈ order)": rates
})
display_dataframe_to_user("FEM Convergence Table", df)
df.head(10)


In [ ]:

# Log–log plot of L2 error vs mesh size h
plt.figure()
plt.loglog(hvals, err, "-d", linewidth=2)
plt.xlabel(r"$h$")
plt.ylabel(r"$\|u-u_h\|_{L^2(0,1)}$")
plt.title("P1-FEM L2 error vs mesh size")
plt.grid(True, which="both")
plt.axis([1e-4, 1, 1e-9, 1e-1])
plt.gca().set_aspect('equal', adjustable='box')
plt.show()
